In [2]:
# https://medium.com/@tomar.ankur287/user-user-collaborative-filtering-recommender-system-51f568489727

import pandas as pd
import numpy as np
import math

Ratings = pd.read_csv('data/parsed_combined_data_1.txt')



In [3]:
Ratings.columns = ['movieId', 'userId', 'rating', 'timestamp']

In [4]:
Ratings

,movieId,userId,rating,timestamp
0,1,822109,5,2005-05-13
1,1,885013,4,2005-10-19
2,1,30878,4,2005-12-26
3,1,823519,3,2004-05-03
4,1,893988,3,2005-11-17
...,...,...,...,...
24053758,4499,2591364,2,2005-02-16
24053759,4499,1791000,2,2005-02-10
24053760,4499,512536,5,2005-07-27
24053761,4499,988963,3,2005-12-20


In [5]:
Mean= Ratings.groupby(['userId'], as_index = False, sort = False).mean().rename(columns = {'rating': 'rating_mean'})[['userId','rating_mean']]
Ratings = pd.merge(Ratings,Mean,on = 'userId', how = 'left', sort = False)
Ratings['rating_adjusted']=Ratings['rating']-Ratings['rating_mean']
Ratings


,movieId,userId,rating,timestamp,rating_mean,rating_adjusted
0,1,822109,5,2005-05-13,4.083333,0.916667
1,1,885013,4,2005-10-19,3.873563,0.126437
2,1,30878,4,2005-12-26,3.634304,0.365696
3,1,823519,3,2004-05-03,3.917197,-0.917197
4,1,893988,3,2005-11-17,3.190141,-0.190141
...,...,...,...,...,...,...
24053758,4499,2591364,2,2005-02-16,3.067729,-1.067729
24053759,4499,1791000,2,2005-02-10,3.972973,-1.972973
24053760,4499,512536,5,2005-07-27,4.085366,0.914634
24053761,4499,988963,3,2005-12-20,3.594340,-0.594340


In [6]:
distinct_movie = np.unique(Ratings['movieId'])
distinct_users = np.unique(Ratings['userId'])


In [7]:
def get_movie_rating(movie_id, user_id):
    user_data_append=pd.DataFrame()
    user_data_all=pd.DataFrame()

    user1_data =  Ratings[Ratings['userId']==user_id]
    user1_mean = user1_data["rating"].mean()
    user1_data = user1_data.rename(columns={'rating_adjusted':'rating_adjusted1'})
    user1_data = user1_data.rename(columns={'userId':'userId1'})
    user1_val = np.sqrt(np.sum(np.square(user1_data['rating_adjusted1']), axis=0))

    movie = movie_id

    item_user =  Ratings[Ratings['movieId']==movie]

    distinct_users1 = np.unique(item_user['userId'])


    j = 1
    
    print('distinct_users1: ' + str(len(distinct_users1)))

    for user2 in distinct_users1:
        user2_data = Ratings[Ratings['userId']==user2]
        user2_data = user2_data.rename(columns={'rating_adjusted':'rating_adjusted2'})
        user2_data = user2_data.rename(columns={'userId':'userId2'})
        user2_val = np.sqrt(np.sum(np.square(user2_data['rating_adjusted2']), axis=0))

        user_data = pd.merge(user1_data,user2_data[['rating_adjusted2','movieId','userId2']],on = 'movieId', how = 'inner', sort = False)
        user_data['vector_product'] = (user_data['rating_adjusted1']*user_data['rating_adjusted2'])

        user_data = user_data.groupby(['userId1','userId2'], as_index = False, sort = False).sum()

        user_data['dot'] = user_data['vector_product']/(user1_val*user2_val)

        user_data_all = pd.concat([user_data_all, user_data], ignore_index=True)

        j = j + 1 
        if j % 100 == 0:
            print('user_comparison: ' + str(j))
    
    print('user comparison done')
    user_data_all =  user_data_all[user_data_all['dot']<1]
    user_data_all = user_data_all.sort_values(['dot'], ascending=False)
    user_data_all = user_data_all.head(30)
    user_data_all['movieId'] = movie
    user_data_append = pd.concat([user_data_append, user_data_all], ignore_index=True)
 
    
    User_dot_adj_rating_all = pd.DataFrame()

    user_data_append_movie = user_data_append[user_data_append['movieId']==movie]
    User_dot_adj_rating = pd.merge(Ratings,user_data_append_movie[['dot','userId2','userId1']], how = 'inner',left_on='userId', right_on='userId2', sort = False)

    User_dot_adj_rating1 = User_dot_adj_rating[User_dot_adj_rating['movieId']==movie]

    if len(np.unique(User_dot_adj_rating1['userId'])) >= 2:

        User_dot_adj_rating1['wt_rating'] = User_dot_adj_rating1['dot']*User_dot_adj_rating1['rating_adjusted']

        User_dot_adj_rating1['dot_abs'] = User_dot_adj_rating1['dot'].abs()
        User_dot_adj_rating1 = User_dot_adj_rating1.groupby(['userId1'], as_index = False, sort = False).sum()[['userId1','wt_rating','dot_abs']]
        User_dot_adj_rating1['Rating'] = (User_dot_adj_rating1['wt_rating']/User_dot_adj_rating1['dot_abs'])+user1_mean
        User_dot_adj_rating1['movieId'] = movie
        User_dot_adj_rating1 = User_dot_adj_rating1.drop(['wt_rating', 'dot_abs'], axis=1)

        User_dot_adj_rating_all = pd.concat([User_dot_adj_rating_all, User_dot_adj_rating1], ignore_index=True)

    User_dot_adj_rating_all = User_dot_adj_rating_all.sort_values(['Rating'], ascending=False)
    return User_dot_adj_rating_all

In [9]:
prediction = pd.DataFrame()
i = 0
with open('data/parsed_qualifying.txt', 'r') as f:
    for l in f.readlines():
        row = l.strip().split(',')
        print(row)
        user_movie_rating = get_movie_rating(movie_id=int(row[0]), user_id=int(row[1]))
        print(user_movie_rating)
        pd.concat([prediction, user_movie_rating])
        i = i + 1
        if i > 1:
            break

prediction
        

['1', '1046323', '2005-12-19']
distinct_users1: 546
user_comparison: 100
user_comparison: 200
user_comparison: 300
user_comparison: 400
user_comparison: 500
user comparison done


/var/folders/qk/mkh7x25n30d8p6y5cmkbt86m0000gn/T/ipykernel_16885/582722326.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_dot_adj_rating1['wt_rating'] = User_dot_adj_rating1['dot']*User_dot_adj_rating1['rating_adjusted']
/var/folders/qk/mkh7x25n30d8p6y5cmkbt86m0000gn/T/ipykernel_16885/582722326.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_dot_adj_rating1['dot_abs'] = User_dot_adj_rating1['dot'].abs()


   userId1    Rating  movieId
0  1046323  4.035139        1
['1', '1080030', '2005-12-23']
distinct_users1: 546
user_comparison: 100
user_comparison: 200
user_comparison: 300
user_comparison: 400
user_comparison: 500
user comparison done
   userId1    Rating  movieId
0  1080030  4.222604        1


/var/folders/qk/mkh7x25n30d8p6y5cmkbt86m0000gn/T/ipykernel_16885/582722326.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_dot_adj_rating1['wt_rating'] = User_dot_adj_rating1['dot']*User_dot_adj_rating1['rating_adjusted']
/var/folders/qk/mkh7x25n30d8p6y5cmkbt86m0000gn/T/ipykernel_16885/582722326.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_dot_adj_rating1['dot_abs'] = User_dot_adj_rating1['dot'].abs()


""
